### Build a Question Answering application over a Graph Database - Neo4j

In [30]:
NEO4J_URI="neo4j+s://1a96045f.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="xe1V7gRacJCYwy0TzDK1RERN8ZLfb5PDzfyhpwH8qzY"

In [31]:
import os
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [32]:
from langchain_community.graphs import Neo4jGraph

In [33]:
# Connect to graph
graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

# Test connection
print(" Connected to Neo4j!")
print("Schema:", graph.schema)

 Connected to Neo4j!
Schema: Node properties:
CEO {POB: STRING, name: STRING, YOB: INTEGER}
company {name: STRING}
enterpreneur {POB: STRING, name: STRING, YOB: INTEGER}
country {name: STRING}
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


In [34]:
## Dataset Moview 
moview_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))


"""

In [35]:
moview_query

"\nLOAD CSV WITH HEADERS FROM\n'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row\n\nMERGE(m:Movie{id:row.movieId})\nSET m.released = date(row.released),\n    m.title = row.title,\n    m.imdbRating = toFloat(row.imdbRating)\nFOREACH (director in split(row.director, '|') | \n    MERGE (p:Person {name:trim(director)})\n    MERGE (p)-[:DIRECTED]->(m))\nFOREACH (actor in split(row.actors, '|') | \n    MERGE (p:Person {name:trim(actor)})\n    MERGE (p)-[:ACTED_IN]->(m))\nFOREACH (genre in split(row.genres, '|') | \n    MERGE (g:Genre {name:trim(genre)})\n    MERGE (m)-[:IN_GENRE]->(g))\n\n\n"

In [36]:
graph.query(moview_query)

[]

In [37]:
graph.refresh_schema()
print(graph.schema)

Node properties:
CEO {POB: STRING, name: STRING, YOB: INTEGER}
company {name: STRING}
enterpreneur {POB: STRING, name: STRING, YOB: INTEGER}
country {name: STRING}
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


In [38]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROK_API_KEY")

In [39]:
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key,model="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020E22A4CE00>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020E22AA43E0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
from langchain.chains import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(
    graph=graph,
    llm=llm,
    verbose=True,
    allow_dangerous_requests=True  # This line is required
)


In [41]:
response=chain.invoke({"query":"Who was the director of the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Casino"})<-[:DIRECTED]-(p:Person) RETURN p.name

Full Context:
[{'p.name': 'Martin Scorsese'}]

> Finished chain.


{'query': 'Who was the director of the movie Casino',
 'result': 'Martin Scorsese was the director of the movie Casino. \n'}

In [45]:
response=chain.invoke({"query":"Who were the actors of the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title:"Casino"})-[:ACTED_IN]->(p:Person) RETURN p.name  

Full Context:
[]

> Finished chain.


{'query': 'Who were the actors of the movie Casino',
 'result': "I don't know the answer. \n"}

In [43]:
response=chain.invoke({"query":"How many artists are there?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person) RETURN count(p) AS artistCount
Full Context:
[{'artistCount': 1239}]

> Finished chain.


{'query': 'How many artists are there?',
 'result': 'There are 1239 artists. \n'}

In [44]:
response=chain.invoke({"query":"How many movies has Tom Hanks acted in"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Tom Hanks"})-[:ACTED_IN]->(m:Movie)
RETURN count(m)

Full Context:
[{'count(m)': 2}]

> Finished chain.


{'query': 'How many movies has Tom Hanks acted in',
 'result': "I don't know the answer. \n"}